In [104]:
import csv
file=open("1488511748129645_facebook_statuses.csv", "r")
reader = csv.reader(file)
data = []
for line in reader:
    data.append(line)

header = data[0]
data = data[1:]

In [105]:
to_remove = []

# simple check to verify the song is on youtube
for row in data:
    if (not 'youtu' in row[6]):
        to_remove.append(row)

print (len(to_remove), len(data))
# print (to_remove)
# row[3][1:-1]

333 4617


In [106]:
youtube_data = [a for a in data if a not in to_remove]
len(youtube_data)

4284

In [107]:
# print (youtube_data)

In [108]:
# Get video id
def parse_yt_url(url):
    if("youtube.com/watch?v" in url):
        return url.split("?v=")[1][:11]
    elif("youtube.com/attribution_link?" in url):
        return url.split("%3D")[1][:11]
    elif("v=" in url):
        return url.split("v=")[-1][:11]
    elif ("youtu.be" in url):
        return url.split("/")[-1][:11]
    else:
        return "-1"

In [109]:
songs_dict = {} # contains 'video id' as key and number of times it has been posted as value
unparsed = []
for row in youtube_data:
    parsed = parse_yt_url(row[6])
    if (not parsed == "-1"):
        try:
            songs_dict[parsed] += 1
        except Exception as e:
            songs_dict[parsed] = 1
    else:
        unparsed.append(row)

In [110]:
len(songs_dict)

3854

In [111]:
len(unparsed)

18

In [112]:
len(youtube_data)

4284

In [113]:
import operator
sorted_songs = sorted(songs_dict.items(), key=operator.itemgetter(1))

In [114]:
# Top N posted songs
N = 10
for song in reversed(sorted_songs[-N:]):
    print ("www.youtube.com/watch?v=" + str(song[0]), " ", song[1])

www.youtube.com/watch?v=fV4DiAyExN0   7
www.youtube.com/watch?v=msuf-V1inOY   5
www.youtube.com/watch?v=adV8-_hgL4g   5
www.youtube.com/watch?v=4zLfCnGVeL4   5
www.youtube.com/watch?v=zLAhRiUeJ8E   4
www.youtube.com/watch?v=4Js-XbNj6Tk   4
www.youtube.com/watch?v=u9Dg-g7t2l4   4
www.youtube.com/watch?v=Kp7eSUU9oy8   4
www.youtube.com/watch?v=pBkHHoOIIn8   4
www.youtube.com/watch?v=8Pa9x9fZBtY   4


In [115]:
for row in youtube_data:
    break

In [116]:
print (youtube_data[2])
youtube_data[1]

['1488511748129645_1918799288434220', 'Total Eargasm.', 'Prachurjya Biswas', "'1749888931717491'", '1. Freed - Danny Elfman | Fifty Shades Freed (Original Motion Picture Score)', 'video', 'https://www.youtube.com/attribution_link?a=a_uH1y1HAII&u=%2Fwatch%3Fv%3DUd2EK0ZslwI%26feature%3Dshare', '2018-02-15 13:11:09', '3', '1', '0', '1', '1', '1', '0', '0', '0', '0', "[{u'type': u'LIKE', u'id': u'428514417569231', u'name': u'Barun Das'}, {u'type': u'LOVE', u'id': u'1832761360102062', u'name': u'Ayushi Shakya'}, {u'type': u'WOW', u'id': u'1814822068551638', u'name': u'Nikhil Jha'}]"]


['1488511748129645_1919067208407428',
 'new Frank Ocean',
 'Siddhant Singh',
 "'1988535971173821'",
 'Frank Ocean - Moon River',
 'video',
 'https://www.youtube.com/watch?v=mXiFHDfvn4A',
 '2018-02-16 03:33:50',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 "[{u'type': u'LIKE', u'id': u'845605628964457', u'name': u'Chandan Suthar'}]"]

In [117]:
# Think of it as new code disjoint from previous one
songs_dict = {} # 'video id' -> [[user_id, reaction_type]]
unparsed = []
for row in youtube_data:
    parsed = parse_yt_url(row[6])
    if (not parsed == "-1"):
        try:
            songs_dict[parsed].append([str(row[3][1:-1]), 'POST'])
            for user in eval(row[-1]):
                songs_dict[parsed].append([user['id'], user['type']])
        except Exception as e:
            songs_dict[parsed] = []
            songs_dict[parsed].append([str(row[3][1:-1]), 'POST'])
            for user in eval(row[-1]):
                songs_dict[parsed].append([user['id'], user['type']])
    else:
        unparsed.append(row)

In [118]:
songs_dict[list(songs_dict.keys())[-1]]

[['2038906389654139', 'POST'],
 ['579882772353326', 'LOVE'],
 ['1600809053334804', 'LOVE'],
 ['1203591703105498', 'LIKE'],
 ['2043020769244845', 'LOVE'],
 ['2000116843540042', 'POST'],
 ['2058593340832566', 'LIKE'],
 ['588968598108888', 'LIKE'],
 ['1486387654805028', 'LIKE'],
 ['1782906748400178', 'LIKE'],
 ['10209024899378554', 'LIKE'],
 ['1477100355728943', 'LIKE']]

In [119]:
users = set()
for dic in songs_dict:
   for val in songs_dict[dic]:
      users.add(val[0])

In [120]:
users_list = list(users)
len(users_list)

1549

In [121]:
songs_list = list(songs_dict.keys())

In [122]:
songs_vec = []
songs_vec

[]

In [123]:
import numpy as np
type_val = {'POST':3, 'LOVE':2, 'ANGRY':-2}
for i in range(len(songs_list)):
    tmp = [0]*len(users_list)
    for user, type_ in songs_dict[songs_list[i]]:
        if (type_ in type_val):
            tmp[users_list.index(user)] = type_val[type_]
        else:
            tmp[users_list.index(user)] = 1
    songs_vec.append(tmp)

In [124]:
def dot(K, L):
   if len(K) != len(L):
      return 0
   return sum(i[0] * i[1] for i in zip(K, L))

# from scipy import spatial
# def dot(K, L):
#    if len(K) != len(L):
#       return 0
#    return 1 - spatial.distance.cosine(K, L)

In [125]:
songs_list.index('u9Dg-g7t2l4')

2071

In [126]:
for song in sorted_songs[-10:]:
    INDEX = songs_list.index(song[0])
    max_song = ''
    max_sim = -1
    for i in range(len(songs_vec)):
        if not i == INDEX:
            tmp = dot(songs_vec[i],songs_vec[INDEX])
            if(tmp>max_sim):
                max_sim = tmp
                max_song = i
    print (song[0], songs_list[max_song], max_sim)            

8Pa9x9fZBtY yuFI5KSPAt4 30
pBkHHoOIIn8 3mbBbFH9fAg 12
Kp7eSUU9oy8 4Js-XbNj6Tk 19
u9Dg-g7t2l4 4zLfCnGVeL4 18
4Js-XbNj6Tk Kp7eSUU9oy8 19
zLAhRiUeJ8E VgwcPiCjQ-0 20
4zLfCnGVeL4 ycYewhiaVBk 26
adV8-_hgL4g 3mbBbFH9fAg 29
msuf-V1inOY xwtdhWltSIg 19
fV4DiAyExN0 n1WpP7iowLc 23


ycYewhiaVBk 4zLfCnGVeL4 26


In [128]:
import numpy as np
songs_vec = np.array(songs_vec)

In [129]:
songs_vec.shape

(3854, 1549)

In [137]:
song = ['ycYewhiaVBk', 42]
ID = '2007257082847724'
INDEX = users_list.index(ID)
max_user = ''
max_sim = -1
for i in range(songs_vec.shape[1]):
    if not i == INDEX:
        tmp = dot(songs_vec[:,i],songs_vec[:,INDEX])
        if(tmp>max_sim):
            max_sim = tmp
            max_user = i
print (ID, users_list[max_user], max_sim)  

2007257082847724 10210714238075480 66


In [153]:
def top_N_sim_song(song_id, N):
    INDEX = songs_list.index(song_id)
    sim_songs = {}
    for i in range(len(songs_vec)):
        tmp = dot(songs_vec[i],songs_vec[INDEX])
        sim_songs[songs_list[i]] = tmp
    returned_list = []
    count = 0
    for w in sorted(sim_songs, key=sim_songs.get, reverse=True):
        returned_list.append([w, sim_songs[w]])
        count += 1
        if (count == N):
            break
    return returned_list[1:]

In [154]:
t = top_N_sim_song('ycYewhiaVBk', 5)

In [155]:
print (t)

[['ycYewhiaVBk', 70], ['4zLfCnGVeL4', 26], ['0UHwkfhwjsk', 21], ['BClzBQmZZBc', 17], ['fj-10lIrboM', 17]]
